# Auto Sampler script
## Elveflow Instruments
* OB1 pressure controller
* MUX Wire
* 
## troubleshooting
* when ob1 set pressure is not working, try: 
 * restarting kernel
 * turn off switches on ob1 and unplug replug usb

## Import Libraries

In [ ]:
%reset -f

# Import Elveflow library
import sys
import os
from email.header import UTF8
# SDK_HOME = 'C:/Users/Admin/ELVEFLOW/SDK_V3_04_04'
SDK_HOME = 'C:/Users/Admin/ELVEFLOW/ESI_V3_04_01/SDK V3_04_01'
sys.path.append(SDK_HOME+'/DLL64/DLL64')#add the path of the library here
sys.path.append(SDK_HOME+'/Python_64')#add the path of the LoadElveflow.py

from ctypes import *

from array import array

from Elveflow64 import *
import serial

from time import sleep, time
import numpy as np
import matplotlib.pyplot as plt

## Instrument Class definitions

In [ ]:
from sampler import *

poe = sampler(verbose=False)

poe.set_valves('air', 'flow-cell')
poe.ob1.set_pressure(0)


In [ ]:
samples = ['tflask1', 'tflask3']
# samples = ['tflask3']
poe.take_measurement(samples=samples, N=10, Nclean=2)

In [ ]:
poe.clean_cell(cycles=1)

In [ ]:
poe.set_valves('water', 'flow-cell')
poe.pushpull(pressure=-50, timeout=10)

In [ ]:
poe.clear(target='allflasks')

In [ ]:
poe.set_valves('air', 'flow-cell')
poe.clear(target='flow-cell', timeout=10, pressure=70)

In [ ]:
poe.clear(target='tflask1', timeout=10)

In [ ]:
poe.to_flowcell(source='tflask3')

In [ ]:
poe.clean_cell(cycles=1)

In [ ]:
poe.acton.send_trigger(N=5)

In [ ]:
poe.to_flowcell(source='water')

In [ ]:
poe.to_flowcell(source='tflask1')

In [ ]:
poe.to_flowcell(source='water')

In [ ]:
poe.mix(source='tflask4', sink='tflask1', Nplugs=2)

In [ ]:
poe.clean(target='flow-cell')

In [ ]:
poe.clear(target='tflask3', timeout=10)

In [ ]:
poe.set_valves('air', 'flow-cell')

In [ ]:
poe.set_valves('air', 'tflask2')

In [ ]:
poe.ob1.set_pressure(250)

In [ ]:
poe.ob1.set_pressure(0)

In [ ]:
poe.ob1.meas_pressure()

In [ ]:
# Clearing lines
poe.set_valves('air', 'tflask1')
poe.pushpull(pressure=500, timeout=10)
poe.set_valves('air', 'tflask2')
poe.pushpull(pressure=500, timeout=10)


poe.set_valves('water', 'flow-cell')
poe.pushpull(pressure=100, timeout=10, sensor=('msr', 2))

poe.set_valves('air', 'flow-cell')
poe.pushpull(pressure=200, timeout=20, sensor=('msr', 2))

In [ ]:
poe.set_valves('water', 'tflask2')
poe.sample(sensor=("msr", 1), setpoint=-500, trigger=0.1, overtime=10.0, timeout=40)

poe.set_valves('air', 'flow-cell')
poe.sample(sensor=("msr", 2), setpoint=50, press_range=(-70,100), trigger=0.0, overtime=0.0, timeout=50)
# poe.pushpull(pressure=300, timeout=10)

# Clean
# poe.set_valves('water', 'tflask')
# poe.pushpull(pressure=500, timeout=5)
poe.set_valves('air', 'tflask2')
poe.pushpull(pressure=250, timeout=10)

# poe.set_valves('air', 'flow-cell')
# poe.pushpull(pressure=100, timeout=10)

In [ ]:
poe.set_valves('air', 'flow-cell')
poe.pushpull(pressure=50, timeout=30)

In [ ]:
sens = msr()

In [ ]:
del sens

In [ ]:
sens.meas_flowrate(channel=3)

In [ ]:
del sens

In [ ]:
poe.set_valves('air', 'flow-cell')

In [ ]:
poe.set_valves('water', 'flow-cell')

In [ ]:
poe.set_valves('air', 'tflask2')

In [ ]:
poe.set_valves('water', 'tflask')

In [ ]:
poe.set_valves('water', 'flow-cell')

In [ ]:
poe.pushpull(pressure=900, timeout=10)

In [ ]:
poe.ob1.set_pressure(-500)

In [ ]:
poe.ob1.set_pressure(100)

In [ ]:
poe.ob1.set_pressure(0)

In [ ]:
poe.ob1.meas_flowrate()

In [ ]:
poe.msr.meas_flowrate(channel=2)

In [ ]:
poe.ob1.meas_pressure()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

vol = np.array([0.2758,
        0.2753,
        0.2761,
        0.2739,
        0.2761,
        0.2748,
        0.2777,
        0.2782,
        0.2771,
        0.2791])*1e3

print('{} Samples: avg={:6.5g} uL, stdev={:6.5g} uL'.format(vol.shape[0], np.mean(vol), np.std(vol)))

plt.figure()
plt.plot(range(1,11), vol, 'o', label='Data')
plt.plot([1,10], [np.mean(vol)]*2, '-', label='Mean={:.3g} $\pm$ {:.3g} $\mu L$'.format(np.mean(vol), np.std(vol)))
plt.legend()
plt.xlabel('Sample #')
plt.ylabel('$\mu L$')
plt.savefig('')
plt.show()

In [ ]:
from time import sleep, time
import matplotlib.pyplot as plt

poe.set_valves('air', 'tflask')

# PID attempt

Kp = 0.5
Ki = 0.2
Kd = 0.005

setpoint = -200.0 # uL/min

timeout = 10.0 # sec
start_time = time()
cur_time = time()-start_time
err_i = 0.0
err_d = 0.0
err_prev = 0.0
dt = 0.15 # sec

# traces
t_arr = []
x_arr = []
y_arr = []
while cur_time < timeout:
    y,flag = poe.ob1.meas_flowrate()
    if flag == 0:
        err = setpoint - y

        err_i = err_i + err*dt
        err_d = (err - err_prev)/dt

        x = Kp*err + Ki*err_i + Kd*err_d
        
        if x > 1000:
            print('Pressure over range {}'.format(x))
            x = 1000.0
        elif x < -900:
            print('Pressure under range {}'.format(x))
            x = -900.0
            
        poe.ob1.set_pressure(x)
        
        x_arr.append(x)
        y_arr.append(y)
        t_arr.append(cur_time)
        

        err_prev = err
        cur_time = time()-start_time
        print('Time {:5.5g}: err={:5.5g} err_i={:5.5g} y={:5.5g} x={:5.5g}'.format(cur_time, err, err_i, y, x))
    else:
        print('Flow rate sensor error {}, value {}'.format(flag, y))
        
    sleep(dt)

poe.set_valves('air', 'tflask')
poe.ob1.set_pressure(200.0)
sleep(5.0)
poe.ob1.set_pressure(0.0)


# plot results
plt.figure(figsize=(10,3))
plt.subplot(121)
plt.title('Pressure [mbar]')
plt.plot(t_arr, x_arr, label='x')
plt.xlabel('Time [sec]')


plt.subplot(122)
plt.title('Y: flowrate [uL/min]')
plt.plot(t_arr, y_arr, label='y')
plt.plot([t_arr[0],t_arr[-1]], [setpoint, setpoint], label='setpoint')
plt.xlabel('Time [sec]')
plt.legend()
plt.savefig('PID.png', dpi=200)
plt.show()

In [ ]:
print('{:4.3g}'.format(1.1122))

In [ ]:
import matplotlib.pyplot as plt
poe.set_valves('air', 'tflask')
time,vol = poe.pushpull(mode=1, target=30, pressure=-20)
poe.set_valves('air', 'flow-cell')
time_rel = [t-time[0] for t in time]
plt.plot(time_rel, vol)
plt.xlabel('Time [sec]')
plt.ylabel('Volume [uL]')
plt.show()

In [ ]:
del poe

In [ ]:
# kenobi=ob1(calibrate=True)
kenobi=ob1(calibrate=False)

In [ ]:
p,e = kenobi.meas_pressure()

print(p)

In [ ]:
kenobi.set_pressure(-50)

In [ ]:
kenobi.meas_flowrate()

In [ ]:
mux = mux_wire()

In [ ]:
state = [1, 0, 0, 0, 0, 0, 0, 0]
mux.set_all_valves(state)

In [ ]:
isinstance(state, list)

In [ ]:
print(os.path.expanduser(r'./ob1_calibration.txt'))
print(os.getcwd())
print(os.path.relpath('ob1_calibration.txt'))
print(os.path.abspath('ob1_calibration.txt'))

In [ ]:
del kenobi
del mux

In [ ]:
print([0]*8)